## workshop for ljy

In [0]:
df_pit_stop = spark.read.csv('s3://columbia-gr5069-main/raw/lap_times.csv', header=True)

In [0]:
display(df_pit_stop)

In [0]:
df_driver = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv',header=True)
df_driver.count()

In [0]:
display(df_driver)

### Transform data

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, current_date
from pyspark.sql.types import IntegerType  # Import IntegerType

df_driver = df_driver.withColumn("age", datediff(current_date(),df_driver.dob)/365)

df_driver = df_driver.withColumn('age', df_driver['age'].cast(IntegerType()))

In [0]:
display(df_driver)

In [0]:
df_lap_drivers = df_driver.join(df_laptimes, on = ['driverId'])



In [0]:
display(df_lap_drivers)

In [0]:
df_lap_drivers = df_driver.select('driverId','nationality','age').join(df_laptimes, on = ['driverId'])


In [0]:
display(df_lap_drivers)

### agggregate age


In [0]:
from pyspark.sql.functions import col, avg
df_lap_drivers = df_lap_drivers.groupBy('nationality','age').agg(avg('milliseconds'))


In [0]:
display(df_lap_drivers)

#### storing data in S3

In [0]:
df_lap_drivers.write.csv('s3://jl6530-gr5069/processed//inclass/laptimes_by_drivers.csv')

In [0]:
import boto3
import pandas as pd

In [0]:
s3 = boto3.client('s3')

In [0]:
df_pit_stop = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header = True)

In [0]:
bucket = "s3://columbia-gr5069-main/raw/"

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, current_date
from pyspark.sql.types import IntegerType 
from pyspark.sql.functions import col, avg, sum, count, when, isnan, lit, round
from pyspark.sql.functions import col, upper, when, substring, regexp_replace, translate


In [0]:
pip install mlflow

In [0]:
import mlflow


In [0]:


df_pit_stop = df_pit_stop.withColumn("driverId", col("driverId").cast(IntegerType()))
df_pit_stop = df_pit_stop.withColumn("raceId", col("raceId").cast(IntegerType()))

# 重命名df_pit_stop中的'milliseconds'列
df_pit_stop = df_pit_stop.withColumnRenamed("milliseconds", "pitStopMilliseconds")


# 重命名df_pit_stop中的'time'列（如果需要）
df_pit_stop = df_pit_stop.withColumnRenamed("time", "pitStopTime")







In [0]:
display(df_pit_stop)

In [0]:

selected_columns = ['driverId', 'raceId',  'stop', 'duration']

df_final_selected = df_pit_stop.select(*selected_columns)


In [0]:
# 可能需要安装MLflow
# pip install mlflow

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tempfile
import os


In [0]:
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 首先确保 `duration` 列存在并转换为数值类型
df_final_selected = df_final_selected.withColumn("durationSeconds", regexp_extract(col("duration"), "(\d+\.\d+)", 1).cast("float"))

# 由于之前的操作已经在 PySpark DataFrame 上执行，现在可以安全地转换为 Pandas DataFrame
pdf_final_selected = df_final_selected.toPandas()

# 准备用于训练的数据
X = pdf_final_selected.drop(['durationSeconds'], axis=1)
y = pdf_final_selected['durationSeconds'].astype(float)  # 确保 durationSeconds 是浮点类型

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 查找和替换 '\\N' 值
X_train = X_train.replace('\\N', np.nan)
y_train = y_train.replace('\\N', np.nan)

X_test = X_test.replace('\\N', np.nan)
y_test = y_test.replace('\\N', np.nan)
# 处理缺失值
# 选项 1: 移除包含缺失值的行
X_train = X_train.dropna()
y_train = y_train.dropna()

X_test = X_test.dropna()
y_test = y_test.dropna()
# 选择 X_train 中的数值型列并转换为 float 类型
X_train_numeric = X_train.select_dtypes(include=[np.number])
X_train = X_train_numeric.astype(float)

X_test_numeric = X_test.select_dtypes(include=[np.number])
X_test = X_test_numeric.astype(float)
# 将 y_train 转换为 float 类型
y_train = y_train.astype(float)
y_test = y_test.astype(float)



In [0]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

with mlflow.start_run(run_name="Basic RF Experiment") as run:
  # Create model, train it, and create predictions
  rf = RandomForestRegressor()
  rf.fit(X_train, y_train)
  predictions = rf.predict(X_test)
  
  # Log model
  mlflow.sklearn.log_model(rf, "random-forest-model")
  
  # Create metrics
  mse = mean_squared_error(y_test, predictions)
  print("  mse: {}".format(mse))
  
  # Log metrics
  mlflow.log_metric("mse", mse)
  
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id
  
  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))

In [0]:
def log_rf(experimentID, run_name, params, X_train, X_test, y_train, y_test):
  import os
  import matplotlib.pyplot as plt
  import mlflow.sklearn
  import seaborn as sns
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
  import tempfile
  import pandas as pd

  with mlflow.start_run(experiment_id=experimentID, run_name=run_name) as run:
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]

    # Create metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("  mse: {}".format(mse))
    print("  mae: {}".format(mae))
    print("  R2: {}".format(r2))

    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)

    # Create feature importance
    importance = pd.DataFrame(list(zip(X_train.columns, rf.feature_importances_)),
                              columns=["Feature", "Importance"]
                             ).sort_values("Importance", ascending=False)

    # Log importances using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="feature-importance-", suffix=".csv", delete=False)
    temp_name = temp.name
    try:
      importance.to_csv(temp_name, index=False)
      mlflow.log_artifact(temp_name, "feature-importance.csv")
    finally:
      temp.close()  # Delete the temp file

    # Create plot for residuals
    fig, ax = plt.subplots()
    sns.residplot(x=predictions, y=y_test - predictions, lowess=True)
    plt.xlabel("Predicted values")
    plt.ylabel("Residuals")
    plt.title("Residual Plot")

    # Log residuals using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="residuals-", suffix=".png", delete=False)
    temp_name = temp.name
    try:
      fig.savefig(temp_name)
      mlflow.log_artifact(temp_name, "residuals.png")
    finally:
      temp.close()  # Delete the temp file

    return run.info.run_uuid


In [0]:
params = {
  "n_estimators": 100,
  "max_depth": 6,
  "random_state": 42
}

log_rf(experimentID, "Second Run", params, X_train, X_test, y_train, y_test)